In [ ]:
"""
Created on Tue Aug 30 15:01 2022

Compute difference between param and reference point by point for melt near grounding line for Figure F1

@author: Clara Burgard
"""

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import cartopy
import cartopy.crs as ccrs
import matplotlib as mpl
import pandas as pd 
from tqdm.notebook import trange, tqdm
import time
import basal_melt_param.melt_functions as meltf
import basal_melt_param.plume_functions as pf
import basal_melt_param.analysis_functions as ana
import basal_melt_param.useful_functions as uf
from basal_melt_param.constants import *
import seaborn as sns
import os, glob
import distributed

In [ ]:
print(mpl.get_backend())

In [ ]:
%matplotlib qt5

READ IN DATA

In [ ]:
home_path='/bettik/burgardc/'


In [ ]:
nemo_run = 'OPM021'

In [ ]:
inputpath_data=home_path+'/DATA/BASAL_MELT_PARAM/interim/NEMO_eORCA025.L121_'+nemo_run+'_ANT_STEREO/'
inputpath_mask = home_path+'/DATA/BASAL_MELT_PARAM/interim/ANTARCTICA_IS_MASKS/nemo_5km_'+nemo_run+'/'
inputpath_profiles = home_path+'/DATA/BASAL_MELT_PARAM/interim/T_S_PROF/nemo_5km_'+nemo_run+'/'
outputpath_simple = home_path+'/DATA/BASAL_MELT_PARAM/interim/SIMPLE/nemo_5km_'+nemo_run+'/'
outputpath_simple_all = home_path+'/DATA/BASAL_MELT_PARAM/interim/SIMPLE/nemo_5km_06161821_oneFRIS/'
inputpath_plumes = home_path+'/DATA/BASAL_MELT_PARAM/interim/PLUMES/nemo_5km_'+nemo_run+'/'
outputpath_plumes = home_path+'/DATA/BASAL_MELT_PARAM/interim/PLUMES/nemo_5km_'+nemo_run+'/'
inputpath_boxes = home_path+'/DATA/BASAL_MELT_PARAM/interim/BOXES/nemo_5km_'+nemo_run+'/'
outputpath_boxes = home_path+'/DATA/BASAL_MELT_PARAM/interim/BOXES/nemo_5km_'+nemo_run+'/'
outputpath_melt = home_path+'/DATA/BASAL_MELT_PARAM/processed/MELT_RATE/nemo_5km_'+nemo_run+'/'
outputpath = home_path+'/DATA/BASAL_MELT_PARAM/interim/'
plot_path = home_path+'/PLOTS/first-look/'

In [ ]:
# make the domain a little smaller to make the computation even more efficient - file isf has already been made smaller at its creation
map_lim = [-3000000,3000000]

In [ ]:
file_other = xr.open_dataset(inputpath_data+'corrected_draft_bathy_isf.nc')#, chunks={'x': chunk_size, 'y': chunk_size})
file_other_cut = uf.cut_domain_stereo(file_other, map_lim, map_lim)
file_conc = xr.open_dataset(inputpath_data+'isfdraft_conc_Ant_stereo.nc')
file_conc_cut = uf.cut_domain_stereo(file_conc, map_lim, map_lim)

In [ ]:
#file_isf_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_new.nc')
file_isf_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_new_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
file_isf = file_isf_nonnan.sel(Nisf=large_isf)
file_isf['isf_name'] = file_isf['isf_name'].astype(str)
file_isf['region'] = file_isf['region'].astype(str)


file_TS_orig = xr.open_dataset(inputpath_profiles+'T_S_mean_prof_corrected_km_contshelf_and_offshore_1980-2018_oneFRIS.nc')
file_TS = file_TS_orig.sel(Nisf=large_isf).sel(profile_domain=[50,1000])

In [ ]:
lon = file_isf.longitude
lat = file_isf.latitude

In [ ]:
file_isf_conc = file_conc_cut['isfdraft_conc']

xx = file_isf.x
yy = file_isf.y
dx = (xx[2] - xx[1]).values
dy = (yy[2] - yy[1]).values
grid_cell_area = abs(dx*dy)  
grid_cell_area_weighted = file_isf_conc * grid_cell_area

In [ ]:
ice_draft_pos = file_other_cut['corrected_isfdraft']
ice_draft_neg = -ice_draft_pos

In [ ]:
isf_stack_mask = uf.create_stacked_mask(file_isf['ISF_mask'], file_isf.Nisf, ['y','x'], 'mask_coord')

In [ ]:
box_charac_all_2D = xr.open_dataset(inputpath_boxes + 'nemo_5km_boxes_2D_oneFRIS.nc')
box_charac_all_1D = xr.open_dataset(inputpath_boxes + 'nemo_5km_boxes_1D_oneFRIS.nc')
plume_charac = xr.open_dataset(inputpath_plumes+'nemo_5km_plume_characteristics_oneFRIS.nc')

In [ ]:
param_var_of_int_2D = file_isf[['ISF_mask', 'latitude', 'longitude', 'dGL']]
param_var_of_int_1D = file_isf[['front_bot_depth_avg', 'front_bot_depth_max','isf_name']]

In [ ]:
geometry_info_2D = plume_charac.merge(param_var_of_int_2D).merge(ice_draft_pos).rename({'corrected_isfdraft':'ice_draft_pos'}).merge(grid_cell_area_weighted).rename({'isfdraft_conc':'grid_cell_area_weighted'}).merge(file_isf_conc)
geometry_info_1D = param_var_of_int_1D

In [ ]:
box_loc_config2 = box_charac_all_2D['box_location'].sel(box_nb_tot=box_charac_all_1D['nD_config'].sel(config=2))
box1 = box_loc_config2.where(box_loc_config2==1).isel(Nisf=0).drop('Nisf')

REFERENCE

In [ ]:
NEMO_melt_rates_2D = xr.open_dataset(outputpath_melt+'melt_rates_2D_NEMO.nc')
ref_box_1 = NEMO_melt_rates_2D['melt_m_ice_per_y'].where(box1==1).mean('time')

SIMPLE PARAMS

In [ ]:
CV_type = 'time'
#CV_type = 'shelves'

In [ ]:
inputpath_chunk_info = '/bettik/burgardc/DATA/BASAL_MELT_PARAM/interim/T_S_PROF/'
info_file = pd.read_csv(inputpath_chunk_info+'info_chunks.txt',header=None, index_col=0)

In [ ]:
gammas_CV_simple = xr.open_dataset(outputpath_simple_all+'gammas_simple_CV_'+CV_type+'.nc')

In [ ]:
simple_param_types = ['linear_local', 'quadratic_local', 'quadratic_local_locslope', 'quadratic_local_cavslope', 
                      'quadratic_mixed_mean', 'quadratic_mixed_locslope','quadratic_mixed_cavslope'] 
#simple_param_types = ['quadratic_local'] 

nisf_list = geometry_info_1D.Nisf

for mparam in simple_param_types:
    
    if CV_type == 'shelves':

        
        ds_2D_all_box1 = None
        for kisf in tqdm(nisf_list):
            #print('ISF: ',kisf.values)
            T_S_profile = file_TS.ffill(dim='depth').sel(Nisf=[kisf])
            
            gamma = gammas_CV_simple['slope'].sel(param=mparam).sel(isf=kisf).drop('isf')
            
            ds_2D, _ = meltf.calculate_melt_rate_1D_and_2D_all_isf([kisf], T_S_profile, geometry_info_2D, geometry_info_1D, isf_stack_mask, mparam, gamma, U_param=True,                                           
                                                           options_2D=['melt_m_ice_per_y'],
                                                           options_1D=['melt_Gt_per_y_tot'], verbose=False)
            
            
            if ds_2D_all_box1 is None:
                ds_2D_all_box1 = ds_2D.where(box1==1).mean('time').copy()
            else:
                ds_2D_all_box1 = ds_2D_all_box1.combine_first(ds_2D.where(box1==1).mean('time'))

        diff_param_ref_CVisf = ds_2D_all_box1['melt_m_ice_per_y'] - ref_box_1
        
        diff_param_ref_CVisf.to_netcdf(outputpath_melt+'diff_melt_param_ref_box1_'+mparam+'_oneFRIS_CVshelves.nc')

    elif CV_type == 'time':

        
        ds_2D_box1_list = []
        for tblock in range(1,14):
            print('TBLOCK: ',tblock)

            tblock_run, tblock_start, tblock_end = info_file.loc[tblock]
            if tblock_run == nemo_run:
                T_S_profile = file_TS.ffill(dim='depth').sel(time=range(tblock_start, tblock_end+1))
            
                gamma = gammas_CV_simple['slope'].sel(param=mparam).sel(tblock=tblock).drop('tblock')
                
                ds_2D, _ = meltf.calculate_melt_rate_1D_and_2D_all_isf(nisf_list, T_S_profile, geometry_info_2D, geometry_info_1D, isf_stack_mask, mparam, gamma, U_param=True,                                           
                                               options_2D=['melt_m_ice_per_y'],
                                               options_1D=['melt_Gt_per_y_tot'])

                ds_2D_box1_list.append(ds_2D.where(box1==1))
            
        ds_2D_box1_CVtime = xr.concat(ds_2D_box1_list, dim='time')
        
        diff_param_ref_CVtime = ds_2D_box1_CVtime['melt_m_ice_per_y'].mean('time') - ref_box_1
    
        diff_param_ref_CVtime.to_netcdf(outputpath_melt+'diff_melt_param_ref_box1_'+mparam+'_oneFRIS_CVtime.nc')


PLUMES

In [ ]:
CV_type = 'shelves'

In [ ]:
plume_param_types = ['lazero19_2','lazero19_modif2'] 

nisf_list = geometry_info_1D.Nisf

for mparam in plume_param_types:
    
    gammas_CV_plumes = xr.open_dataset(outputpath_simple_all + 'plume_gamma_CV'+CV_type+'.nc')
    E0s_CV_plumes = xr.open_dataset(outputpath_simple_all + 'plume_E0_CV'+CV_type+'.nc')
    
    if CV_type == 'shelves':
        
        ds_2D_all_box1 = None
        for kisf in tqdm(nisf_list):
            #print('ISF: ',kisf.values)
            T_S_profile = file_TS.ffill(dim='depth').sel(Nisf=[kisf])
            
            gamma = gammas_CV_plumes['gamma'].sel(param=mparam).sel(Nisf=kisf).drop('Nisf')
            E0 = E0s_CV_plumes['E0'].sel(param=mparam).sel(Nisf=kisf).drop('Nisf')
            
            ds_2D, _ = meltf.calculate_melt_rate_1D_and_2D_all_isf([kisf], T_S_profile, geometry_info_2D, geometry_info_1D, isf_stack_mask, mparam, gamma, E0=E0, 
                                                                   options_2D=['melt_m_ice_per_y'],
                                                                   options_1D=['melt_Gt_per_y_tot'], verbose=False)
            
            
            if ds_2D_all_box1 is None:
                ds_2D_all_box1 = ds_2D.where(box1==1).mean('time').copy()
            else:
                ds_2D_all_box1 = ds_2D_all_box1.combine_first(ds_2D.where(box1==1).mean('time'))

        diff_param_ref_CVisf = ds_2D_all_box1['melt_m_ice_per_y'] - ref_box_1
        
        diff_param_ref_CVisf.to_netcdf(outputpath_melt+'diff_melt_param_ref_box1_'+mparam+'_oneFRIS_CVshelves.nc')

    elif CV_type == 'time':

        
        ds_2D_box1_list = []
        for tblock in range(1,14):
            print('TBLOCK: ',tblock)

            tblock_run, tblock_start, tblock_end = info_file.loc[tblock]
            if tblock_run == nemo_run:
                T_S_profile = file_TS.ffill(dim='depth').sel(time=range(tblock_start, tblock_end+1))
            
                gamma = gammas_CV_plumes['gamma'].sel(param=mparam).sel(tblock=tblock).drop('tblock')
                E0 = E0s_CV_plumes['E0'].sel(param=mparam).sel(tblock=tblock).drop('tblock')
                
                ds_2D, _ = meltf.calculate_melt_rate_1D_and_2D_all_isf(nisf_list, T_S_profile, geometry_info_2D, geometry_info_1D, isf_stack_mask, mparam, gamma, E0=E0, 
                                                                   options_2D=['melt_m_ice_per_y'],
                                                                   options_1D=['melt_Gt_per_y_tot'], verbose=True)

                ds_2D_box1_list.append(ds_2D.where(box1==1))
            
        ds_2D_box1_CVtime = xr.concat(ds_2D_box1_list, dim='time')
        
        diff_param_ref_CVtime = ds_2D_box1_CVtime['melt_m_ice_per_y'].mean('time') - ref_box_1
        
        diff_param_ref_CVtime.to_netcdf(outputpath_melt+'diff_melt_param_ref_box1_'+mparam+'_oneFRIS_CVtime.nc')

BOXES

In [ ]:
CV_type = 'shelves'

In [ ]:
nisf_list = geometry_info_1D.Nisf

gammas_CV_boxes = xr.open_dataset(outputpath_simple_all + 'box_gamma_CV'+CV_type+'.nc')
Cs_CV_boxes = xr.open_dataset(outputpath_simple_all + 'box_C_CV'+CV_type+'.nc')

for nD_config in range(1,5):
    
    pism_version = 'yes'
    picop_opt = 'no'
    mparam = 'boxes_'+str(nD_config)+'_pism'+pism_version+'_picop'+picop_opt
            
    if CV_type == 'shelves':
        
        ds_2D_all_box1 = None
        for kisf in tqdm(nisf_list):
            #print('ISF: ',kisf.values)
            T_S_profile = file_TS.ffill(dim='depth').sel(Nisf=[kisf])
            
            gamma = gammas_CV_boxes['gamma'].sel(param=mparam).sel(Nisf=kisf).drop('Nisf') * 10**(-5)
            C = Cs_CV_boxes['C'].sel(param=mparam).sel(Nisf=kisf).drop('Nisf') * 10**6
            
            ds_2D, _ = meltf.calculate_melt_rate_1D_and_2D_all_isf([kisf], T_S_profile, geometry_info_2D, geometry_info_1D, isf_stack_mask, mparam, gamma,
                                                                   C=C, box_charac_2D=box_charac_all_2D, box_charac_1D=box_charac_all_1D, box_tot=nD_config, box_tot_option='nD_config', 
                                                                   pism_version=pism_version, picop_opt=picop_opt,                                                                    
                                                                   options_2D=['melt_m_ice_per_y'],
                                                                   options_1D=['melt_Gt_per_y_tot'], verbose=False)
            
            
            if ds_2D_all_box1 is None:
                ds_2D_all_box1 = ds_2D.where(box1==1).mean('time').copy()
            else:
                ds_2D_all_box1 = ds_2D_all_box1.combine_first(ds_2D.where(box1==1).mean('time'))

        diff_param_ref_CVisf = ds_2D_all_box1['melt_m_ice_per_y'] - ref_box_1
        
        diff_param_ref_CVisf.to_netcdf(outputpath_melt+'diff_melt_param_ref_box1_'+mparam+'_oneFRIS_CVshelves.nc')

    elif CV_type == 'time':

        
        ds_2D_box1_list = []
        for tblock in range(1,14):
            print('TBLOCK: ',tblock)

            tblock_run, tblock_start, tblock_end = info_file.loc[tblock]
            if tblock_run == nemo_run:
                T_S_profile = file_TS.ffill(dim='depth').sel(time=range(tblock_start, tblock_end+1))
            
                gamma = gammas_CV_boxes['gamma'].sel(param=mparam).sel(tblock=tblock).drop('tblock') * 10**(-5)
                C = Cs_CV_boxes['C'].sel(param=mparam).sel(tblock=tblock).drop('tblock') * 10**6
                
                ds_2D, _ = meltf.calculate_melt_rate_1D_and_2D_all_isf(nisf_list, T_S_profile, geometry_info_2D, geometry_info_1D, isf_stack_mask, mparam, gamma,
                                                                   C=C, box_charac_2D=box_charac_all_2D, box_charac_1D=box_charac_all_1D, box_tot=nD_config, box_tot_option='nD_config', 
                                                                   pism_version=pism_version, picop_opt=picop_opt,                                                                    
                                                                   options_2D=['melt_m_ice_per_y'],
                                                                   options_1D=['melt_Gt_per_y_tot'], verbose=True)

                ds_2D_box1_list.append(ds_2D.where(box1==1))
            
        ds_2D_box1_CVtime = xr.concat(ds_2D_box1_list, dim='time')
        
        diff_param_ref_CVtime = ds_2D_box1_CVtime['melt_m_ice_per_y'].mean('time') - ref_box_1
        
        diff_param_ref_CVtime.to_netcdf(outputpath_melt+'diff_melt_param_ref_box1_'+mparam+'_oneFRIS_CVtime.nc')

PICOP

In [ ]:
tuned_box_gammas = xr.DataArray(np.array([[[0.39e-5,0.41e-5,0.44e-5,0.39e-5],
                                              [0.39e-5,0.41e-5,0.44e-5,0.39e-5]],
                                         [[0.51e-5,0.73e-5,0.92e-5,0.63e-5],
                                              [0.51e-5,0.73e-5,0.92e-5,0.63e-5]]]
                                        ), dims={'profile_domain': np.array([50,1000]), 'pism': ['no','yes'], 'config': range(1,5)})
tuned_box_gammas = tuned_box_gammas.assign_coords({'profile_domain': np.array([50,1000]),'pism': ['no','yes'], 'config': range(1,5)})

tuned_box_Cs = xr.DataArray(np.array([[[16.5e6,18.0e6,20.7e6,20.7e6],
                                              [16.1e6,17.8e6,20.5e6,20.5e6]],
                                    [[0.14e6,0.14e6,0.14e6,0.13e6],
                                              [0.14e6,0.14e6,0.14e6,0.13e6]]]), dims={'profile_domain': np.array([50,1000]), 'pism': ['no','yes'], 'config': range(1,5)})
tuned_box_Cs = tuned_box_Cs.assign_coords({'profile_domain': np.array([50,1000]),'pism': ['no','yes'], 'config': range(1,5)})

In [ ]:
CV_type = 'shelves'

In [ ]:
nisf_list = geometry_info_1D.Nisf

gammas_CV_picop = xr.open_dataset(outputpath_simple_all + 'picop_gamma_CV'+CV_type+'.nc')
E0s_CV_picop = xr.open_dataset(outputpath_simple_all + 'picop_E0_CV'+CV_type+'.nc')

for nD_config in range(3,5):
    
    if nD_config == 3:
        pism_version = 'yes'
    elif nD_config == 4:
        pism_version = 'no'
        
    picop_opt = 'yes'
    mparam = 'boxes_'+str(nD_config)+'_pism'+pism_version+'_picop'+picop_opt
    
    picop_opt = '2019'
    
    gamma = tuned_box_gammas.sel(config=nD_config, pism=pism_version)
    C = tuned_box_Cs.sel(config=nD_config, pism=pism_version)
            
    if CV_type == 'shelves':
        
        ds_2D_all_box1 = None
        for kisf in tqdm(nisf_list):
            #print('ISF: ',kisf.values)
            T_S_profile = file_TS.ffill(dim='depth').sel(Nisf=[kisf])
            
            gamma_plume = gammas_CV_picop['gamma'].sel(param=mparam).sel(Nisf=kisf).drop('Nisf')
            E0 = E0s_CV_picop['E0'].sel(param=mparam).sel(Nisf=kisf).drop('Nisf')
            
            ds_2D, _ = meltf.calculate_melt_rate_1D_and_2D_all_isf([kisf], T_S_profile, geometry_info_2D, geometry_info_1D, isf_stack_mask, mparam, gamma,
                                                                      C=C, E0=E0, angle_option='appenB',
                                                                      box_charac_2D=box_charac_all_2D, box_charac_1D=box_charac_all_1D, box_tot=nD_config, box_tot_option='nD_config', 
                                                                      pism_version=pism_version, picop_opt=picop_opt, gamma_plume=gamma_plume,                                                                    
                                                                      options_2D=['melt_m_ice_per_y'],
                                                                      options_1D=['melt_Gt_per_y_tot'], verbose=False)
            
            
            if ds_2D_all_box1 is None:
                ds_2D_all_box1 = ds_2D.where(box1==1).mean('time').copy()
            else:
                ds_2D_all_box1 = ds_2D_all_box1.combine_first(ds_2D.where(box1==1).mean('time'))

        diff_param_ref_CVisf = ds_2D_all_box1['melt_m_ice_per_y'] - ref_box_1
        
        diff_param_ref_CVisf.to_netcdf(outputpath_melt+'diff_melt_param_ref_box1_'+mparam+'_oneFRIS_CVshelves.nc')

    elif CV_type == 'time':

        
        ds_2D_box1_list = []
        for tblock in range(1,14):
            print('TBLOCK: ',tblock)

            tblock_run, tblock_start, tblock_end = info_file.loc[tblock]
            if tblock_run == nemo_run:
                T_S_profile = file_TS.ffill(dim='depth').sel(time=range(tblock_start, tblock_end+1))
            
                gamma_plume = gammas_CV_picop['gamma'].sel(param=mparam).sel(tblock=tblock).drop('tblock') 
                E0 = E0s_CV_picop['E0'].sel(param=mparam).sel(tblock=tblock).drop('tblock') 
                
                ds_2D, _ = meltf.calculate_melt_rate_1D_and_2D_all_isf(nisf_list, T_S_profile, geometry_info_2D, geometry_info_1D, isf_stack_mask, mparam, gamma,
                                                                      C=C, E0=E0, angle_option='appenB',
                                                                      box_charac_2D=box_charac_all_2D, box_charac_1D=box_charac_all_1D, box_tot=nD_config, box_tot_option='nD_config', 
                                                                      pism_version=pism_version, picop_opt=picop_opt, gamma_plume=gamma_plume,                                                                    
                                                                      options_2D=['melt_m_ice_per_y'],
                                                                      options_1D=['melt_Gt_per_y_tot'], verbose=True)
                

                ds_2D_box1_list.append(ds_2D.where(box1==1))
            
        ds_2D_box1_CVtime = xr.concat(ds_2D_box1_list, dim='time')
        
        diff_param_ref_CVtime = ds_2D_box1_CVtime['melt_m_ice_per_y'].mean('time') - ref_box_1
        
        diff_param_ref_CVtime.to_netcdf(outputpath_melt+'diff_melt_param_ref_box1_'+mparam+'_oneFRIS_CVtime.nc')